# Benchmarks
---

## Line optical depth
--- 

In [1]:
import matplotlib.pyplot as plt
import numpy             as np
import torch

from p3droslo.model import TensorModel
from p3droslo.lines import Line
from astropy        import constants
from time           import time
from ipywidgets     import interact

In [2]:
R = 495000.0
N = 128

nH2 = 1.0e+12        # [m^-3]
nCO = 1.0e-4 * nH2   # [m^-3]
T   = 45             # [k]
trb = 150.0          # [m/s]
cΔβ = 0.35

model = TensorModel(shape=(N,), sizes=(R,))
model['nCO'        ]  = nCO * np.ones(N)
model['temperature']  = T   * np.ones(N)
model['velocity_z']   = cΔβ * np.linspace(0.0, R, N)
model['v_turbulence'] = trb * np.ones(N)

print('velocity increment =', cΔβ * model.dx(0), 'm/s')

velocity increment = 1353.515625 m/s


In [3]:
line = Line(
    species_name = "test",
    transition   = 0,
    datafile     = "data/test.txt",
    molar_mass   = 1.0
)

You have selected line:
    test(J=-)
Please check the properties that were inferred:
    Frequency         1.798754700e+11  Hz
    Einstein A coeff  1.000000000e-04  1/s
    Molar mass        1.0              g/mol


In [4]:
N_freqs = 100
v_pixel = 5000.0

dd    = (N_freqs-1)/2 * v_pixel/constants.c.si.value
fmin  = line.frequency - line.frequency*dd
fmax  = line.frequency + line.frequency*dd
freqs = torch.linspace(fmin, fmax, N_freqs, dtype=torch.float64)

In [5]:
def get_doppler_shifted_frequencies(v_los, frequencies):
    """
    Doppler shifts frequencies given the velocity along the line of sight.
    """
    # Compute the Doppler shift for each cell
    shift = 1.0 + v_los * (1.0 / constants.c.si.value)

    # Create freqency tensor for each cell
    freqs = torch.einsum("..., f -> ...f", shift, frequencies)
    
    return freqs

In [6]:
nCO    = model['nCO']        
tmp    = model['temperature']
vel    = model['velocity_z']
v_turb = model['v_turbulence']
    
frequencies = get_doppler_shifted_frequencies(vel, freqs)
    
# Compute the LTE line emissivity and opacity, and the line profile
eta, chi = line.LTE_emissivity_and_opacity(nCO, tmp, v_turb, frequencies)

tau_old = torch.empty_like(chi) 
tau_old[...,  0 , :] = 0.0
tau_old[..., +1:, :] = torch.cumsum(chi, dim=0)[..., :-1, :] * model.dx(0)

In [7]:
pop     = line.LTE_pops(temperature=tmp)
chi_ij  = line.opacity_ij(pop=pop)

In [8]:
tau_new = line.optical_depth(
    chi_ij       = chi_ij,
    density      = nCO,
    temperature  = tmp,
    v_turbulence = v_turb,
    velocity_los = vel,
    frequencies  = freqs,
    dx           = model.dx(0)
)

In [9]:
%%timeit
tau_old = torch.empty_like(chi) 
tau_old[...,  0 , :] = 0.0
tau_old[..., +1:, :] = torch.cumsum(chi, dim=0)[..., :-1, :] * model.dx(0)

96.6 µs ± 528 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
%%timeit
tau_new = line.optical_depth(
    chi_ij       = chi_ij,
    density      = nCO,
    temperature  = tmp,
    v_turbulence = v_turb,
    velocity_los = vel,
    frequencies  = freqs,
    dx           = model.dx(0)
)

4.07 ms ± 88.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
def plot(i):
    plt.figure(dpi=150)
    plt.plot(tau_old[:,i])
    plt.plot(tau_new[:,i])
    plt.yscale('log')
    plt.show()

interact(plot, i=(0,N_freqs-1))

interactive(children=(IntSlider(value=49, description='i', max=99), Output()), _dom_classes=('widget-interact'…

<function __main__.plot(i)>

In [12]:
def plot(i):
    plt.figure(dpi=150)
    plt.plot(tau_old[i,:])
    plt.plot(tau_new[i,:])
    plt.show()

interact(plot, i=(0,N-1))

interactive(children=(IntSlider(value=63, description='i', max=127), Output()), _dom_classes=('widget-interact…

<function __main__.plot(i)>

In [13]:
from scipy.special import erf

# line width
δν = line.gaussian_width(model['temperature'], model['v_turbulence'])[0].item()

# Compute the prefactor
factor = constants.h.si.value * line.frequency / (4.0 * np.pi)

# Compute the LTE level populations
pop = line.LTE_pops(model['temperature'])

# Compute the emissivity and opacity
chi  = factor * (line.Einstein_Ba * pop[line.lower] - line.Einstein_Bs * pop[line.upper])
chi *= model['nCO']
chi  = chi[0].item()

In [14]:
abs_rel_diff = lambda a,b: np.abs(2.0*(a-b)/(a+b))

The optical depth is given by,
\begin{equation}
\begin{split}
\tau(\nu)
\ &= \
\int_{0}^{\ell} \text{d} l \ \chi_{ij} \, \phi \big( (1+\Delta\beta l) \nu \big) \\
\ &= \
\frac{\chi_{ij}}{2 \nu \Delta \beta}
\left(
    \text{Erf}\left[ \frac{\left(1 + \Delta\beta \ell \right) \nu - \nu_{ij}}{\delta\nu_{ij}} \right]
    \ - \
    \text{Erf}\left[ \frac{\nu - \nu_{ij}}{\delta\nu_{ij}} \right]
\right) .
\end{split}
\end{equation}

In [15]:
# Analytic solution
ν     = freqs
fac   = 1.0 + model['velocity_z'] / constants.c.si.value
fac_ν = torch.einsum("i, f -> if",  fac, ν)

τ = constants.c.si.value * chi/(2.0*ν*cΔβ) * (erf((fac_ν-line.frequency)/δν) - erf((ν-line.frequency)/δν) )

In [16]:
def plot2(i):
    fig, ax = plt.subplots(2,1, dpi=250)
    ax[0].plot(tau_old[i,:])
    ax[0].plot(tau_new[i,:])
    ax[0].plot(      τ[i,:])
    ax[1].plot(abs_rel_diff(tau_old[i,:], τ[i,:]))
    ax[1].plot(abs_rel_diff(tau_new[i,:], τ[i,:]))
    ax[1].set_yscale('log')
    plt.show()

interact(plot2, i=(0,N-1))

interactive(children=(IntSlider(value=63, description='i', max=127), Output()), _dom_classes=('widget-interact…

<function __main__.plot2(i)>